# Initialize

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from qick import QickConfig
import sys 
sys.path.append('../')
from slab_qick_calib.exp_handling.instrumentmanager import InstrumentManager
#import slab_qick_calib.experiments as meas
import slab_qick_calib.config as config

%load_ext autoreload
%autoreload 2

# Configuration
ip = '10.108.30.23'

expt_path = 'C:\\_Data\\Terracat\\2025-04-28\\'
cfg_file='Terracat1.yml'

# Set seaborn color palette
colors = ["#0869c8", "#b51d14", '#ddb310', '#658b38', '#7e1e9c', '#75bbfd', '#cacaca']
sns.set_palette(sns.color_palette(colors))

# Figure parameters
plt.rcParams['figure.figsize'] = [8, 4]
plt.rcParams.update({'font.size': 13})

print('Data will be stored in', expt_path)

# Results config file
cfg_path = os.path.join(os.getcwd(), 'configs', cfg_file)
auto_cfg = config.load(cfg_path)

# Connect to instruments
im = InstrumentManager(ns_address=ip)
print(im)
soc = QickConfig(im[auto_cfg['aliases']['soc']].get_cfg())
print(soc)

cfg_dict = {'soc': soc, 'expt_path': expt_path, 'cfg_file': cfg_path, 'im': im}


## New directory

In [ ]:
import os
import slab_qick_calib.config as config
new_config = False
new_folder = False
expt_path = 'C:\\_Data\\Terracat\\2025-04-28\\'
cfg_file='Terracat1.yml'

if new_config or new_folder:

    if new_config:
        config.init_config( os.getcwd() + '\\configs\\' + cfg_file, 8, type='full', aliases='bf1_soc', t1=30)

    if not os.path.exists(expt_path):
        os.makedirs(expt_path)
        os.mkdir(expt_path + 'images')
        os.mkdir(expt_path + 'images\\summary')

## TOF

In [ ]:
qubit_list = np.arange(3)
qubit_list = [0]
for qi in qubit_list:  
        tof=meas.ToFCalibrationExperiment(cfg_dict=cfg_dict, qi=qi, params={'frequency':7000})
        #tof=meas.ToFCalibrationExperiment(cfg_dict=cfg_dict, qi=qi,params={'readout_length':13})#,params={'frequency':fi})
        # use this to set trig_offset in config file

In [ ]:
tof=meas.ToF2D(cfg_dict=cfg_dict, qi=qi)


In [ ]:
tof.data['i'].shape

## Coarse

In [ ]:
#rspec = cfg.make_rspec_coarse(soc, expt_path, cfg_path, i, im=im, start=6000, span=800, reps=2000, npts=5000)
qi=0
rspecc = meas.ResSpec(cfg_dict, qi=qi, style='coarse', params={'start':6000, 'span':2100, 'rounds':1, 'reps':15, 'gain':1, 'expts':100000,'length':100})
res_values = rspecc.data['coarse_peaks']

In [ ]:
rspecc.analyze(peaks=True, debug=True, fit=False, prom=0.05)
rspecc.display(peaks=True, fit=False)
res_values = rspecc.data['coarse_peaks']

In [ ]:
res_values = np.delete(res_values,1)

## Res fine

In [ ]:
update=False

qubit_list = np.arange(8)
qubit_list=[5]
for qi in qubit_list:    
    rspec = meas.ResSpec(cfg_dict, qi=qi, params={'span':0.1, 'center':res_values[qi], 'gain':1,'reps':4,'length':500}) # Use this to sweep through the res_values list
    #rspec = meas.ResSpec(cfg_dict, qi=qi, params={'span':'kappa','reps':50,'gain':1,'length':500})
    #rspec = meas.ResSpec(cfg_dict, qi=qi, params={'span':0.1,'long_pulse':True,'gain':1,'length':500})
    if update: rspec.update(cfg_dict['cfg_file'])

In [ ]:
plt.figure()
fix_phase = np.unwrap(rspec.data['phases']-rspec.data['phases'][0])-fit_params[0]*rspec.data['xpts']
amps = rspec.data['amps']
x = amps*np.cos(fix_phase)
y = amps*np.sin(fix_phase)
plt.plot(x, y, 'o', label='raw data')

## Power sweep

In [ ]:
qi=4
qi_list=[]
gains = [1, 0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125, 0.00390625, 0.001953125]
#gains = [0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125, 0.00390625, 0.001953125]
d=[]
for i, g in enumerate(gains):
    reps = int(np.ceil(4*(2)**i))
    print(reps)
    rspec = meas.ResSpec(cfg_dict, qi=qi, params={'span':'kappa','long_pulse':True,'reps':reps,'gain':g,'expts':100})
    rspec.update(cfg_dict['cfg_file'])
    d.append(rspec.data['amps'])
    qi_list.append(rspec.data['fit'][1])

plt.figure()
plt.plot(qi_list,'-o')
plt.title(f'Resonator {qi}')

In [ ]:
plt.figure()

for i, dd in enumerate(d): 
    plt.semilogy(dd*2**i)

In [ ]:
plt.figure()
plt.semilogy(np.array(qi_list[1:-1])*10/1000,'-o')
plt.title(f'Resonator {qi}')

# Check that reps don't change anygthing 

In [ ]:
qi=0
qi_list=[]
reps = [1,2,5,10,20,50]
for i, r in enumerate(reps):

    rspec = meas.ResSpec(cfg_dict, qi=qi, params={'span':'kappa','long_pulse':True,'reps':r,'gain':1,'expts':100})
    rspec.update(cfg_dict['cfg_file'])
    qi_list.append(rspec.data['fit'][1])

plt.figure()
plt.plot(qi_list,'-o')
plt.title(f'Resonator {qi}')

In [ ]:
qi=0
qi_list=[]
length = [50,100,200,500,1000,2000,5000,10000]
reps = [100,50,20,10,5,2,1,1]
for i, r in enumerate(reps):

    rspec = meas.ResSpec(cfg_dict, qi=qi, params={'span':'kappa','long_pulse':True,'reps':r,'gain':1,'expts':100,'length':length[i]})
    rspec.update(cfg_dict['cfg_file'])
    qi_list.append(rspec.data['fit'][1])

plt.figure()
plt.plot(qi_list,'-o')
plt.title(f'Resonator {qi}')

# Get data for the phase change

In [ ]:

rspec = meas.ResSpec(cfg_dict, qi=qi, params={'span':1,'long_pulse':True,'reps':1,'gain':1,'expts':10000,'center':6033})


In [ ]:
plt.figure()
phases = np.unwrap(rspec.data['phases'][300:])
x = rspec.data['xpts'][300:]
fit_params = np.polyfit(x, phases, 1)
fit_line = np.polyval(fit_params, x)

plt.plot(phases, label='Unwrapped Phases')
plt.plot(fit_line, label='Linear Fit', linestyle='--')
plt.legend()

fit_params[0]
#-1130.8379223790125
